In [0]:
# 99_utils/99_reset_kardia_environment.ipynb
# Wipes all Kardia-related databases and DBFS folders to start clean.

import re, time
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()

# Stop any running streams
for q in spark.streams.active:
    print(f"Stopping stream {q.id}...")
    q.stop()
    q.awaitTermination(30)

# Drop project databases
for db in spark.catalog.listDatabases():
    if db.name.startswith("kardia_"):
        print(f"Dropping database: {db.name}")
        spark.sql(f"DROP DATABASE {db.name} CASCADE")

# Delete all related DBFS folders
folders = [
    "dbfs:/kardia/raw/",
    "dbfs:/kardia/bronze/",
    "dbfs:/kardia/silver/",
    "dbfs:/kardia/gold/",
    "dbfs:/kardia/_schemas/",
    "dbfs:/kardia/_checkpoints/",
    "dbfs:/kardia/_quarantine/",
    "dbfs:/kardia/_state/"
]

for path in folders:
    try:
        dbutils.fs.rm(path, recurse=True)
        print(f"Deleted: {path}")
    except Exception as e:
        print(f"Could not delete {path}: {e}")

print("Kardia environment reset complete.")
